# Day 16: Ticket Translation

https://adventofcode.com/2020/day/16

In [190]:
# Can I simplfy this? I'm sure there's a way to inline the string->int loop, but or now this works...

def readInput(filename):
    with open(filename) as f:
        blocks = [ f.split("\n") for f in [ b for b in f.read().split("\n\n") ] ]
    rules = {}
    for b in blocks[0]:
        r = b.split(": ")[0]
        l = [ i.split("-") for i in b.split(": ")[1].split(" or ") ]
        r0 = []
        for i in l:
            f0 = []
            for ii in i:
                f0.append(int(ii))
            r0.append(f0)
        rules[r] = r0
    myticket = [ int(n) for n in blocks[1][1].split(",") ]
    tt = [b.split(",") for b in blocks[2][1:-1]]
    tickets = []
    for t in tt:
        tickets.append([ int(i) for i in t ])
    return rules,myticket,tickets

In [191]:
rules,myticket,tickets = readInput("data/day16test0.txt")
print(rules)
print(myticket)
print(tickets)

{'class': [[1, 3], [5, 7]], 'row': [[6, 11], [33, 44]], 'seat': [[13, 40], [45, 50]]}
[7, 1, 14]
[[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]]


## Part 1

In [195]:
def checkRule(n,r):
    return r[0][0] <= n <= r[0][1] or r[1][0] <= n <= r[1][1]

def checkTicket(t,rules):
    vt = 0
    for n in t:
        vr = False
        for r in rules.keys():
            if checkRule(n,rules[r]):
                vr = True
                break
        if not vr:
            return False, n
        else:
            vt += 1
            continue
    return True, -1
            
def checkTickets(tickets,rules):
    val = []
    failn = []
    for t in tickets:
        v,n = checkTicket(t,rules)
        val.append(v)
        failn.append(n)
    fn = np.array(failn)
    return fn[fn>-1].sum(), val, failn

In [196]:
rules,myticket,tickets = readInput("data/input16.txt")
result,val,failn = checkTickets(tickets,rules)
print(result)

25916


## Part 2

In [197]:
# Filter invalid tickets

from itertools import compress
tvalid = list(compress(tickets,val))

len(tickets),len(tvalid),len(tvalid[0]),len(rules)

(243, 190, 20, 20)

In [198]:
# Part 2 example, all tickets are valid

rules,myticket,tickets = readInput("data/day16test1.txt")
result,val,failn = checkTickets(tickets,rules)
print(val)

[True, True, True]
